In [1]:
import tkinter as tk # for create GUI
from tkinter import simpledialog, messagebox # foe input dialog and message
import pandas as pd # for handle CSV data
from rapidfuzz import fuzz, process # for fuzzy string matching

# define the CalorieApp class, inheriting from tkinter's Tk class to create the main window
class CalorieApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title('Calorie Calculator')
        self.geometry('400x600') # set the size of the window

        # initialize total calories and eaten calories
        self.total_calories_goal = 1300 # daily calorie goal,adjustable
        self.eaten_calories = 0 # variable to track the number of calories eaten
        self.burned_calories = 300 # static value for now

        # top sevtion with calories left and burned
        self.top_frame = tk.Frame(self) # create a frame to hold the top section
        self.top_frame.pack(pady=10) # for better spacing

        # label to show eaten calories
        self.eaten_label = tk.Label(self.top_frame, text=f'{self.eaten_calories} eaten', font=('Arial', 16))
        self.eaten_label.grid(row=0, column=0, padx =20) # place it in the grid at row 0, column 0

        # create a frame to display the calories left
        self.cal_left_frame = tk.Frame(self.top_frame, width=100, height=100)
        self.cal_left_frame.grid(row=0, column=1)
        # label to display tje calories left = total goal - eaten calories
        self.cal_left_label = tk.Label(self.cal_left_frame, text=f'{self.total_calories_goal - self.eaten_calories}\ncal left', font=('Ariel', 18), fg="black", width=10, height=3)
        self.cal_left_label.grid(row=0, column=1) # palce it in the middle of the top frame

        # label to show burned calories
        self.burned_label = tk.Label(self.top_frame, text=f"{self.burned_calories} burned", font=("Arial", 16))
        self.burned_label.grid(row=0, column=2, padx=20)

        # macronutrients row (carbs, protein, fat)
        self.macros_frame = tk.Frame(self)  # create a frame to hold the macronutrient labels
        self.macros_frame.pack(pady=10)

        # label for carbs
        self.carbs_label = tk.Label(self.macros_frame, text="carbs", font=("Arial", 14))
        self.carbs_label.grid(row=0, column=0, padx=20)  # place it in the grid at row 0, column 0

        # label for protein
        self.protein_label = tk.Label(self.macros_frame, text="protein", font=("Arial", 14))
        self.protein_label.grid(row=0, column=1, padx=20) 

        # label for fat
        self.fat_label = tk.Label(self.macros_frame, text="fat", font=("Arial", 14))
        self.fat_label.grid(row=0, column=2, padx=20) 

        # meal buttons (breakfast, lunch, dinner, snack)
        self.meals_frame = tk.Frame(self)  # create a frame to hold the meal buttons
        self.meals_frame.pack(pady=10)

        # button for breakfast, calls search_food with "breakfast" as the argument
        self.breakfast_button = tk.Button(self.meals_frame, text="☕ breakfast", font=("Arial", 14), command=lambda: self.search_food("breakfast"))
        self.breakfast_button.grid(row=0, column=0, pady=10, ipadx=10) 

        # button for lunch, calls search_food with "lunch" as the argument
        self.lunch_button = tk.Button(self.meals_frame, text="🍱 lunch", font=("Arial", 14), command=lambda: self.search_food("lunch"))
        self.lunch_button.grid(row=1, column=0, pady=10, ipadx=10) 

        # button for dinner, calls search_food with "dinner" as the argument
        self.dinner_button = tk.Button(self.meals_frame, text="🥗 dinner", font=("Arial", 14), command=lambda: self.search_food("dinner"))
        self.dinner_button.grid(row=2, column=0, pady=10, ipadx=10)

        # button for snack, calls search_food with "snack" as the argument
        self.snack_button = tk.Button(self.meals_frame, text="🍎 snack", font=("Arial", 14), command=lambda: self.search_food("snack"))
        self.snack_button.grid(row=3, column=0, pady=10, ipadx=10) 

        # home button, can be used for future navigation
        self.home_button = tk.Button(self, text="🏠", font=("Arial", 18))
        self.home_button.pack(pady=20)

        # read the calorie data drom the CSV file
        self.df = pd.read_csv('calories.csv')

    # function to perform fuzzy search on the 'FoodItem' column
    def fuzzy_find_calories(self, search_term, n=1):
        matches = process.extract(search_term, self.df['FoodItem'], scorer=fuzz.token_set_ratio, limit=n)
        return matches[0] # get the first(best) match
    
    # function to handle food search for each meal
    def search_food(self, meal):
        search_term = simpledialog.askstring('input', f'Enter food item for {meal}:') # ask user for the food item to search for
        if not search_term:
            return # if no input, exit the fuction
        
        # search the FoodItem column in the DataFrame df for the search term with the fuzzy search
        food_item, match_score, index = self.fuzzy_find_calories(search_term)

        # check if the match score is 100 or if user wants to lower the threshold
        if match_score >= 100:
            self.process_food_selection(food_item, index)

        else:
            # if no exact match (100), automatically check for 50 score match
            if match_score >= 50:
                self.process_food_selection(food_item, index)
            else:
                messagebox.showinfo("No Match", f"No suitable match found for {search_term}!")  # show a message if no good match is found

    # function to process the selected food item after matching
    def process_food_selection(self, food_item, index):
        row = self.df.iloc[index]
        # ask the user for the weight or portion size of the food
        weight = simpledialog.askfloat('input', f"Enter the weight in grams for {food_item}: ")
        if weight is None:  # default to 100 grams if no input
            weight = 100

        # calculate total calories based on the portion size
        total_calories = (float(row['Kcals_per100grams']) * weight)/100

        # update the eaten calories and recalculate the calories left
        self.eaten_calories += total_calories
        self.update_calories_display() # update the GUI display with new values

    # function to update the calorie-related labels in the GUI
    def update_calories_display(self):
        self.eaten_label.config(text=f"{self.eaten_calories:.0f} eaten")  # update the eaten calories label
        self.cal_left_label.config(text=f"{self.total_calories_goal - self.eaten_calories:.0f}\ncal left")  # update the calories left label
    
app = CalorieApp()
app.mainloop()

/var/folders/sf/t4kybqzs6v7b5wh0vqj50cm80000gn/T/ipykernel_25216/3951540713.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # for handle CSV data
